# Sudoku solver

I use a simple brute force method:  
1. Find all coordinates without a number (a zero instead of a number 1 to 9).
2. Try for the first coordinate the number 1.
3. Check if number is possible: no nr 1 in row, column or sector present already.
4. If number already present, try higher number for that coordinate.
5. If number is possible in current coordinate, go to next coordinate. 
6. If no number is possible at all in a coordinate, it means we have filled an earlier coordinate with the wrong number. So we backtrack one coordinate and try for that coordinate a different, higher number.

In [1]:
import numpy as np

In [2]:
sudoku1 = np.array([
    [0, 7, 8, 0, 0, 0, 1, 3, 0],
    [6, 4, 0, 7, 1, 0, 0, 0, 0],
    [0, 0, 0, 8, 5, 4, 0, 2, 7],
    [4, 0, 5, 9, 0, 0, 7, 0, 0],
    [0, 0, 0, 2, 0, 1, 0, 8, 3],
    [0, 8, 3, 0, 0, 6, 0, 1, 9],
    [0, 6, 9, 0, 0, 0, 2, 0, 0],
    [0, 0, 4, 1, 0, 5, 0, 7, 0],
    [1, 0, 0, 3, 0, 2, 9, 0, 8]
])

In [3]:
sudoku2 = np.array([
    0, 0, 0, 7, 0, 1, 0, 0, 0,
    5, 0, 0, 0, 3, 0, 2, 0, 4,
    0, 0, 0, 0, 0, 0, 1, 0, 5,
    0, 4, 0, 0, 0, 0, 0, 0, 0,
    6, 0, 1, 5, 7, 2, 0, 0, 0,
    0, 0, 7, 0, 4, 6, 0, 0, 0,
    9, 0, 0, 0, 0, 0, 0, 2, 0,
    0, 0, 3, 0, 1, 8, 0, 5, 0,
    0, 0, 0, 0, 2, 5, 3, 0, 8
]).reshape(9, 9)

In [4]:
hard_sudoku = np.array([
    9, 4, 0, 0, 2, 0, 7, 0, 0, 
    0, 0, 1, 0, 0, 4, 0, 0, 9, 
    0, 0, 6, 0, 0, 0, 1, 2, 0, 
    0, 0, 0, 0, 0, 3, 0, 1, 0,
    1, 0, 0, 0, 0, 0, 0, 0, 8,
    0, 7, 0, 5, 0, 0, 0, 0, 0,
    0, 8, 7, 0, 0, 0, 2, 0, 0,
    6, 0, 0, 9, 0, 0, 3, 0, 0,
    0, 0, 9, 0, 8, 0, 0, 5, 7
]).reshape(9, 9)

In [5]:
def is_valid_entry(nr, row, col, matrix):
    """
    Check if the number is already in the row, column or sector.
    If so then the number is not a valid solution and returns False.
    Otherwise if the number is a valid entry, the function returns True.
    """
    
    def already_in_row():
        """Returns whether the matrix row already contains this number"""
        return np.isin(nr, matrix[row, :])
    
    def already_in_column():
        """Returns whether the matrix column already contains this number"""
        return np.isin(nr, matrix[:, col])
    
    def already_in_sector():
        """Returns whether the matrix sector already contains this number"""
        start_row = (row // 3) * 3
        end_row = start_row + 3
        
        start_col = (col // 3) * 3
        end_col = start_col + 3
        current_sector = matrix[start_row:end_row, start_col:end_col]
        
        return np.isin(nr, current_sector)
    
    if nr > 9 or already_in_row() or already_in_column() or already_in_sector():
        return False
    else: 
        return True

In [6]:
def get_coordinates_that_need_solution(matrix):
    """Return all the coordinates that have a 0 in the sudoku and need a solution."""
    return np.dstack(np.where(matrix==0))[0]

In [7]:
def find_solutions(matrix):
    """
    Returns a solution for a sudoku matrix.
    """
    
    # find all coordinates that don't have a number 1 to 9 yet
    coordinates_to_solve = get_coordinates_that_need_solution(matrix)
    
    coord_nr = 0
    backtracks = 0
    
    while coord_nr < len(coordinates_to_solve):
        
        row = coordinates_to_solve[coord_nr, 0]
        col = coordinates_to_solve[coord_nr, 1]

        # start trying number 1 to 9 in current coordinate
        for number in range(matrix[row, col]+1, 11):
            
            valid_entry = is_valid_entry(number, row, col, matrix)

            if valid_entry:
                matrix[row, col] = number
                # go to next coordinate
                coord_nr += 1
                break
            
            # if no number 1 to 9 is possible in current coordinate:
            # it means we have a wrong number in an earlier coordinate, so:
            # set current coordinate to 0 and go one coordinate back.
            if number > 9:
                matrix[row, col] = 0
                coord_nr -= 1
                backtracks += 1
 
    print(f'nr of coordinates to solve: {len(coordinates_to_solve)}',
          f'nr of backtracks: {backtracks}')
    print(matrix)
    
    return matrix

In [8]:
%%timeit -n1 -r1
solution1 = find_solutions(sudoku1.copy())
solution1

nr of coordinates to solve: 43 nr of backtracks: 27
[[5 7 8 6 2 9 1 3 4]
 [6 4 2 7 1 3 8 9 5]
 [9 3 1 8 5 4 6 2 7]
 [4 1 5 9 3 8 7 6 2]
 [7 9 6 2 4 1 5 8 3]
 [2 8 3 5 7 6 4 1 9]
 [3 6 9 4 8 7 2 5 1]
 [8 2 4 1 9 5 3 7 6]
 [1 5 7 3 6 2 9 4 8]]
45.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
%%timeit -n1 -r1
solution2 = find_solutions(sudoku2.copy())
solution2

nr of coordinates to solve: 54 nr of backtracks: 171
[[2 3 4 7 5 1 6 8 9]
 [5 1 8 6 3 9 2 7 4]
 [7 6 9 2 8 4 1 3 5]
 [8 4 2 1 9 3 5 6 7]
 [6 9 1 5 7 2 8 4 3]
 [3 5 7 8 4 6 9 1 2]
 [9 8 5 3 6 7 4 2 1]
 [4 2 3 9 1 8 7 5 6]
 [1 7 6 4 2 5 3 9 8]]
126 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n1 -r1
hard_sudoku_solution = find_solutions(hard_sudoku.copy())
hard_sudoku_solution

nr of coordinates to solve: 55 nr of backtracks: 57221
[[9 4 8 6 2 1 7 3 5]
 [2 3 1 7 5 4 8 6 9]
 [7 5 6 3 9 8 1 2 4]
 [5 6 4 8 7 3 9 1 2]
 [1 9 3 4 6 2 5 7 8]
 [8 7 2 5 1 9 6 4 3]
 [4 8 7 1 3 5 2 9 6]
 [6 2 5 9 4 7 3 8 1]
 [3 1 9 2 8 6 4 5 7]]
29.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
